In [1]:
%matplotlib inline

import numpy as np
import scipy

In [6]:
A = np.random.rand(4,4)
A

array([[0.86493647, 0.60948246, 0.27333156, 0.92490418],
       [0.07221441, 0.56605454, 0.80351532, 0.08408469],
       [0.5234643 , 0.24909628, 0.04634613, 0.35757391],
       [0.26778524, 0.52857144, 0.40008656, 0.37125925]])

In [11]:
scipy.linalg.lu(A)

(array([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.]]),
 array([[ 1.        ,  0.        ,  0.        ,  0.        ],
        [ 0.083491  ,  1.        ,  0.        ,  0.        ],
        [ 0.30960105,  0.65973599,  1.        ,  0.        ],
        [ 0.6052055 , -0.23247912, -0.31273966,  1.        ]]),
 array([[ 0.86493647,  0.60948246,  0.27333156,  0.92490418],
        [ 0.        ,  0.51516824,  0.7806946 ,  0.00686351],
        [ 0.        ,  0.        , -0.1995895 ,  0.08037984],
        [ 0.        ,  0.        ,  0.        , -0.1754496 ]]))